In [ ]:
import json, os
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from weight_estimation.utils import CameraMetadata, get_ann_from_keypoint_arrs, get_left_right_keypoint_arrs, \
    normalize_left_right_keypoint_arrs, convert_to_world_point_arr
from weight_estimation.body_parts import core_body_parts
from weight_estimation.weight_estimator import WeightEstimator


<h1> Load base dataset </h1>

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
    select * from keypoint_annotations
    where keypoints is not null
    and keypoints -> 'leftCrop' is not null
    and keypoints -> 'rightCrop' is not null
    limit 10000;
"""
df = rds_access_utils.extract_from_database(query)

<h1> Add synthetic data </h1>

<h2> Add weight field </h2>

In [ ]:
# weight_model_f = '/root/data/s3/aquabyte-models/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb'
# kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/playground/kf_predictor_v2.pb')

# weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

# weights = []
# for idx, row in df.iterrows():
#     camera_metadata = row.camera_metadata
    
#     camera_metadata_obj = CameraMetadata(
#         focal_length=camera_metadata['focalLength'],
#         focal_length_pixel=camera_metadata['focalLengthPixel'],
#         baseline_m=camera_metadata['baseline'],
#         pixel_count_width=camera_metadata['pixelCountWidth'],
#         pixel_count_height=camera_metadata['pixelCountHeight'],
#         image_sensor_width=camera_metadata['imageSensorWidth'],
#         image_sensor_height=camera_metadata['imageSensorHeight']
#     )
    
#     weight, length, kf = weight_estimator.predict(annotation, camera_metadata_obj)
#     weights.append(weight)


In [ ]:
lengths = []
for idx, row in df.iterrows():
    camera_metadata = row.camera_metadata
    cm = CameraMetadata(
        focal_length=camera_metadata['focalLength'],
        focal_length_pixel=camera_metadata['focalLengthPixel'],
        baseline_m=camera_metadata['baseline'],
        pixel_count_width=camera_metadata['pixelCountWidth'],
        pixel_count_height=camera_metadata['pixelCountHeight'],
        image_sensor_width=camera_metadata['imageSensorWidth'],
        image_sensor_height=camera_metadata['imageSensorHeight']
    )
    
    ann = row.keypoints
    X_left, X_right = get_left_right_keypoint_arrs(ann)
    
    world_keypoints = convert_to_world_point_arr(X_left, X_right, cm)
    length = np.linalg.norm(world_keypoints[6] - world_keypoints[7])
    lengths.append(length)

df['length'] = lengths

<h2> Simulate Larger Fish </h2>

In [ ]:
def simulate_larger_fish(world_keypoints, pct_inflation):
    modified_world_keypoints = {}
    for body_part in core_body_parts:
        kps = world_keypoints[body_part]
        modified_kps = (1.0 + pct_inflation) * kps
        modified_world_keypoints[body_part] = modified_kps
        
    return modified_world_keypoints
    

def get_ann_from_world_keypoints(world_keypoints, cm):
    ann = {'leftCrop': [], 'rightCrop': []}
    for body_part in core_body_parts:
        x, y, z = world_keypoints[body_part]
        px_x = round(x * cm['focalLengthPixel'] / y + cm['pixelCountWidth'] / 2.0)
        px_y = round(cm['pixelCountHeight'] / 2.0 - z * cm['focalLengthPixel'] / y)
        disparity = round(cm['focalLengthPixel'] * cm['baseline'] / y)
        
        left_item = {
            'keypointType': body_part,
            'xFrame': px_x,
            'yFrame': px_y
        }
        
        right_item = {
            'keypointType': body_part,
            'xFrame': px_x - disparity,
            'yFrame': px_y + random.uniform(0, 10)
        }
        
        ann['leftCrop'].append(left_item)
        ann['rightCrop'].append(right_item)
    return ann
    

In [ ]:
modified_anns, cms = [], []

mask = df.length > np.percentile(df.length, 90)
for idx, row in df[mask].iterrows():
    camera_metadata = row.camera_metadata
    cm = CameraMetadata(
        focal_length=camera_metadata['focalLength'],
        focal_length_pixel=camera_metadata['focalLengthPixel'],
        baseline_m=camera_metadata['baseline'],
        pixel_count_width=camera_metadata['pixelCountWidth'],
        pixel_count_height=camera_metadata['pixelCountHeight'],
        image_sensor_width=camera_metadata['imageSensorWidth'],
        image_sensor_height=camera_metadata['imageSensorHeight']
    )
    
    ann = row.keypoints
    X_left, X_right = get_left_right_keypoint_arrs(ann)
    
    world_keypoints = convert_to_world_point_arr(X_left, X_right, cm)
    world_keypoints_dict = {}
    for idx, body_part in enumerate(core_body_parts):
        world_keypoints_dict[body_part] = world_keypoints[idx]
        
    modified_world_keypoints = simulate_larger_fish(world_keypoints_dict, random.uniform(0, 0.25))
    modified_ann = get_ann_from_world_keypoints(modified_world_keypoints, camera_metadata)
    modified_anns.append(modified_ann)
    cms.append(camera_metadata)



In [ ]:
all_anns = list(df.keypoints.values) + modified_anns

<h1> Construct "good" and "bad" class </h1>

In [ ]:
PIXEL_COUNT_WIDTH = 4096


def convert_to_akpd_nn_input(ann):
    X_left, X_right = get_left_right_keypoint_arrs(ann)
    X_left_norm, X_right_norm = normalize_left_right_keypoint_arrs(X_left, X_right)
    X = np.hstack([X_left_norm, X_right_norm]) / PIXEL_COUNT_WIDTH
    return X


def perturb_ann(ann, p_perturbation=0.2, min_magnitude=30, max_magnitude=200):
    
    left_keypoints, right_keypoints = ann['leftCrop'], ann['rightCrop']
    perturbed_left_keypoints = []
    
    # pick body parts to perturb (at least one)
    indices = []
    while len(indices) == 0:
        indices = [x for x in range(len(core_body_parts)) if (random.random() < p_perturbation)]
    
    # apply perturbation
    perturbed_left_keypoints, perturbed_right_keypoints = [], []
    for idx, _ in enumerate(left_keypoints):
        left_item, right_item = left_keypoints[idx], right_keypoints[idx]
        left_perturbation_x, right_perturbation_x, left_perturbation_y, right_perturbation_y = \
            0.0, 0.0, 0.0, 0.0
        if idx in indices:
            case = np.random.choice([0, 1, 2], 1).item()
            if case == 0:
                left_perturbation_x = np.random.normal(0, np.random.uniform(low=min_magnitude, high=max_magnitude))
                right_perturbation_x = np.random.normal(0, np.random.uniform(low=min_magnitude, high=max_magnitude))
                left_perturbation_y = np.random.normal(0, np.random.uniform(low=min_magnitude, high=max_magnitude))
                right_perturbation_y = np.random.normal(0, np.random.uniform(low=min_magnitude, high=max_magnitude))
            elif case == 1:
                x_magnitude = np.random.uniform(low=min_magnitude, high=max_magnitude)
                y_magnitude = np.random.uniform(low=min_magnitude, high=max_magnitude)
                left_perturbation_x = np.random.normal(0, x_magnitude)
                right_perturbation_x = np.random.normal(0, abs(x_magnitude + np.random.normal(0, 20)))
                left_perturbation_y = np.random.normal(0, y_magnitude)
                right_perturbation_y = np.random.normal(0, abs(y_magnitude + np.random.normal(0, 20)))
            else:
                k = list(range(len(core_body_parts)))
                k.remove(idx)
                random_idx = np.random.choice(k, 1).item()
                left_perturbation_x = left_keypoints[random_idx]['xFrame'] - left_item['xFrame'] + np.random.normal(0, 20)
                left_perturbation_y = left_keypoints[random_idx]['yFrame'] - left_item['yFrame'] + np.random.normal(0, 20)
                right_perturbation_x = right_keypoints[random_idx]['xFrame'] - right_item['xFrame'] + np.random.normal(0, 20)
                right_perturbation_y = right_keypoints[random_idx]['yFrame'] - right_item['yFrame'] + np.random.normal(0, 20)

        perturbed_left_item = {
            'keypointType': left_item['keypointType'],
            'xFrame': left_item['xFrame'] + left_perturbation_x,
            'yFrame': left_item['yFrame'] + left_perturbation_y
        }

        perturbed_right_item = {
            'keypointType': right_item['keypointType'],
            'xFrame': right_item['xFrame'] + right_perturbation_x,
            'yFrame': right_item['yFrame'] + right_perturbation_y
        }
        
        perturbed_left_keypoints.append(perturbed_left_item)
        perturbed_right_keypoints.append(perturbed_right_item)

    perturbed_keypoints = {
        'leftCrop': perturbed_left_keypoints,
        'rightCrop': perturbed_right_keypoints
    }
    
    return perturbed_keypoints
        


X_good_arr, X_bad_arr = [], []
count = 0
for ann in all_anns:
    
    # construct "good" class
    
    X_good = convert_to_akpd_nn_input(ann)
    X_good_arr.append(X_good)
    
    # construct "bad" class
    ann_bad = perturb_ann(ann)
    X_bad = convert_to_akpd_nn_input(ann_bad)
    X_bad_arr.append(X_bad)
    

X_good_arr = np.array(X_good_arr)
X_bad_arr = np.array(X_bad_arr)

<h1> Train Model </h1>

<h2> Create train / val split </h2>

In [ ]:
train_pct = 0.8

X_good_train = X_good_arr[:int(len(X_good_arr) * train_pct)]
X_good_val = X_good_arr[int(len(X_good_arr) * train_pct):]
X_bad_train = X_bad_arr[:int(len(X_bad_arr) * train_pct)]
X_bad_val = X_bad_arr[int(len(X_bad_arr) * train_pct):]

X_train = np.vstack([X_good_train, X_bad_train])
y_train = np.array([1] * len(X_good_train) + [0] * len(X_bad_train))
shuffle_idx = np.array(range(len(X_train)))
np.random.shuffle(shuffle_idx)
X_train = X_train[shuffle_idx]
y_train = y_train[shuffle_idx]

X_val = np.vstack([X_good_val, X_bad_val])
y_val = np.array([1] * len(X_good_val) + [0] * len(X_bad_val))
shuffle_idx = np.array(range(len(X_val)))
np.random.shuffle(shuffle_idx)
X_val = X_val[shuffle_idx]
y_val = y_val[shuffle_idx]


X_train = X_train.reshape(len(X_train), -1)
X_val = X_val.reshape(len(X_val), -1)


<h2> Train model </h2>

In [ ]:
import os
import math
import keras
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras.optimizers import RMSprop
from keras.models import load_model


In [ ]:
inputs = Input(shape=(32,))

x = Dense(256, activation='relu')(inputs)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)


In [ ]:
model = Model(inputs=inputs, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
optimizer = RMSprop(lr=0.0001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                               min_delta=0,
                                               patience=100,
                                               verbose=0,
                                               mode='auto')]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callbacks, batch_size=32, epochs=100)

<h1> Test on Real Examples </h1>

In [ ]:
s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))

In [ ]:
def display_crops(left_image_f, right_image_f, left_keypoints, right_keypoints, side='both', overlay_keypoints=True, show_labels=False):
    assert side == 'left' or side == 'right' or side == 'both', \
        'Invalid side value: {}'.format(side)

    if side == 'left' or side == 'right':
        fig, ax = plt.subplots(figsize=(20, 10))
        image_f = left_image_f if side == 'left' else right_image_f
        keypoints = left_keypoints if side == 'left' else right_keypoints
        image = plt.imread(image_f)
        ax.imshow(image)

        if overlay_keypoints:
            for bp, kp in keypoints.items():
                ax.scatter([kp[0]], [kp[1]], color='red', s=1)
                if show_labels:
                    ax.annotate(bp, (kp[0], kp[1]), color='red')
    else:
        fig, axes = plt.subplots(2, 1, figsize=(20, 20))
        left_image = plt.imread(left_image_f)
        right_image = plt.imread(right_image_f)
        axes[0].imshow(left_image)
        axes[1].imshow(right_image)
        if overlay_keypoints:
            for bp, kp in left_keypoints.items():
                axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
                if show_labels:
                    axes[0].annotate(bp, (kp[0], kp[1]), color='red')
            for bp, kp in right_keypoints.items():
                axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
                if show_labels:
                    axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))
query = """
        select * from prod.biomass_computations
        where pen_id=144 and captured_at >= '2021-01-08' and captured_at <= '2021-01-12';
    """
tdf = rds_access_utils.extract_from_database(query)

In [ ]:
akpd_scores = []
count = 0
for idx, row in tdf.iterrows():
    
    ann = row.annotation
    X = convert_to_akpd_nn_input(ann)
    score = model.predict(np.array(X.reshape(-1, 32)))[0][0]
    akpd_scores.append(score)
    
    

In [ ]:
tdf[np.array(akpd_scores) > 0.01].estimated_weight_g.mean()

In [ ]:
tdf[tdf.akpd_score > 0.01].estimated_weight_g.mean()

In [ ]:
idx = 9
left_image_url = tdf.left_crop_url.iloc[idx]
right_image_url = tdf.left_crop_url.iloc[idx]
left_image_f, _, _ = s3.download_from_url(left_image_url)
right_image_f, _, _ = s3.download_from_url(right_image_url)

ann = tdf.annotation.iloc[idx]
left_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['leftCrop']}
right_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['rightCrop']}

display_crops(left_image_f, right_image_f, left_keypoints, right_keypoints)